In [1]:
!python -m pip install requests
!python -m pip install pandas
!python -m pip install tqdm
!python -m pip install pandasql

In [16]:
import requests
import json
import pandas as pd
from tqdm import tqdm

# Carregar o arquivo CSV 'movies.csv' em um DataFrame
df_movies = pd.read_csv('movies.csv', delimiter='|')

# Filtrar apenas os filmes de Ação/Aventura
df_filtered = df_movies[(df_movies['tempoMinutos'] != 'N') &
                        (df_movies['numeroVotos'] >= 5000) &
                        (df_movies['genero'].str.contains('action', case=False) |
                         df_movies['genero'].str.contains('adventure', case=False))]

df_filtered = df_filtered[['id', 'tituloPincipal', 'tempoMinutos', 'anoLancamento', 'genero', 'notaMedia', 'numeroVotos']]
df_filtered = df_filtered.groupby('tituloPincipal').first()
df_filtered = df_filtered.sort_values(by=['notaMedia', 'tempoMinutos'], ascending=[False, True])

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

num_linhas = len(df_filtered)
print("Número de linhas no DataFrame filtrado: ", num_linhas)

print(df_filtered.to_string(index=False))



C:\Users\jadem\AppData\Local\Temp\ipykernel_26304\497186740.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('movies.csv', delimiter='|')


Número de linhas no DataFrame filtrado:  4453
        id tempoMinutos anoLancamento                      genero  notaMedia  numeroVotos
tt17592606          121          2022   Adventure,Biography,Drama        9.4        11536
 tt0259534           97          1993  Action,Adventure,Animation        9.2         8246
 tt0468569          152          2008          Action,Crime,Drama        9.0      2640318
 tt0167260          201          2003      Action,Adventure,Drama        9.0      1838696
 tt0237376          124          1989                Action,Drama        8.9         7803
 tt4139928          153          2022        Action,Drama,Fantasy        8.9         8837
 tt7466810          164          2022      Adventure,Comedy,Drama        8.9        30865
 tt0075143          207          1976  Action,Adventure,Biography        8.9         9474
 tt1375666          148          2010     Action,Adventure,Sci-Fi        8.8      2339961
tt15327088          148          2022      Action,Adve

Foi impresso todos os filmes dentro dos parametros:
Ação ou Aventura
mais de 5000 votos
tempo em minutos não nulo
ordenado por nota
retirou repetições de id

Resultado deu 4453 titulos.

In [25]:
import requests

# Criar uma lista de URLs únicas com base nos IDs dos filmes filtrados
api_key = "XXXXXXXXXXXXXXXXXXXXXXX"
urls = set()

for movie_id in df_filtered['id'].head(20):  # Selecionar os 20 primeiros IDs
    url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&language=pt-BR'
    urls.add(url)

# Criar uma lista para armazenar os dados dos filmes
filmes = []

# Fazer solicitações para as URLs e obter os dados dos filmes
for url in urls:
    response = requests.get(url)
    if response.status_code == 200:
        filme_data = response.json()
        filmes.append(filme_data)

# Mostrar os dados dos filmes e outras informações interessantes
for filme in filmes:
    print("ID:", filme["id"])
    print("Título:", filme["title"])
    print("País de Origem:", filme["production_countries"][0]["name"])
    print("Gêneros:", [genre["name"] for genre in filme["genres"]])
    print("Data de Lançamento:", filme["release_date"])
    print("Resumo:", filme["overview"]) 
    print("Nota Média:", filme["vote_average"])
    print("Número de Votos:", filme["vote_count"])
    print("--------------------")


ID: 464293
Título: The Legend of Maula Jatt
País de Origem: Pakistan
Gêneros: ['Drama', 'Fantasia', 'Ação']
Data de Lançamento: 2022-10-13
Resumo: 
Nota Média: 8.1
Número de Votos: 9
--------------------
ID: 27205
Título: A Origem
País de Origem: United Kingdom
Gêneros: ['Ação', 'Ficção científica', 'Aventura']
Data de Lançamento: 2010-07-15
Resumo: Um ladrão que rouba segredos corporativos por meio do uso da tecnologia de compartilhamento de sonhos, recebe a tarefa inversa de plantar uma ideia na mente de um Diretor Executivo.
Nota Média: 8.363
Número de Votos: 33759
--------------------
ID: 429
Título: Três Homens em Conflito
País de Origem: United States of America
Gêneros: ['Faroeste']
Data de Lançamento: 1966-12-23
Resumo: Durante o auge da Guerra Civil, um misterioso pistoleiro vaga pela fronteira do oeste. Ele não possui um lar, lealdade ou companhia... Até que encontra dois estrangeiros, que são tão brutos e desapegados quanto ele. Unidos pelo destino, os três homens juntam sua

In [3]:
import requests
import json
import pandas as pd
from tqdm import tqdm

# Carregar o arquivo CSV 'movies.csv' em um DataFrame
df_movies = pd.read_csv('movies.csv', delimiter='|')

# Filtrar apenas os filmes de Ação/Aventura
df_filtered = df_movies[(df_movies['tempoMinutos'] != 'N') &
                        (df_movies['numeroVotos'] >= 15000) &
                        (df_movies['genero'].str.contains('action', case=False) &
                         df_movies['genero'].str.contains('adventure', case=False))]

df_filtered = df_filtered[['id', 'tituloPincipal', 'tempoMinutos', 'anoLancamento', 'genero', 'notaMedia', 'numeroVotos']]
df_filtered = df_filtered.groupby('tituloPincipal').first()
df_filtered = df_filtered.sort_values(by=['notaMedia', 'tempoMinutos'], ascending=[False, True])

# Definir lista para armazenar os resultados
filmes = []

# Definir URL base e chave da API
base_url = 'https://api.themoviedb.org/3/movie/'
api_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXX'

# Definir barra de progresso
pbar = tqdm(total=len(df_filtered))

# Iterar sobre os IDs dos filmes filtrados
for movie_id in df_filtered['id']:
    # Construir a URL da API para obter as informações do país de produção do filme
    url = f'{base_url}{movie_id}?api_key={api_key}&language=pt-BR'
    
    # Fazer solicitação para a URL e obter os dados do filme
    response = requests.get(url)
    
    if response.status_code == 200:
        filme_data = response.json()
        
        # Extrair o nome do país de produção do filme
        countries = filme_data.get('production_countries', [])
        production_country = countries[0]['name'] if countries else 'N/A'
        
        # Adicionar a informação do país de produção ao dataframe
        df_filtered.loc[df_filtered['id'] == movie_id, 'production_country'] = production_country
        
        # Adicionar os dados do filme à lista de resultados
        filmes.append(filme_data)
    
    # Atualizar a barra de progresso
    pbar.update(1)
    
    # Parar a iteração após obter os 20 primeiros resultados
    #if len(filmes) >= 20:
    #    break

# Fechar a barra de progresso
pbar.close()

# Imprimir os dados obtidos
print(df_filtered.head(20).to_string(index=False))


C:\Users\jadem\AppData\Local\Temp\ipykernel_7068\2883387899.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('movies.csv', delimiter='|')
100%|████████████████████████████████████████████████████████████████████████████████| 754/754 [05:53<00:00,  2.14it/s]

        id tempoMinutos anoLancamento                     genero  notaMedia  numeroVotos       production_country
 tt0167260          201          2003     Action,Adventure,Drama        9.0      1838696              New Zealand
 tt1375666          148          2010    Action,Adventure,Sci-Fi        8.8      2339961           United Kingdom
tt15327088          148          2022     Action,Adventure,Drama        8.8        75358                    India
 tt0120737          178          2001     Action,Adventure,Drama        8.8      1867920              New Zealand
 tt0167261          179          2002     Action,Adventure,Drama        8.8      1660335              New Zealand
 tt0080684          124          1980   Action,Adventure,Fantasy        8.7      1288030 United States of America
 tt0076759          121          1977   Action,Adventure,Fantasy        8.6      1360570 United States of America
 tt9900782          145          2019     Action,Adventure,Crime        8.5        32631

Tive que limitar para Aventura E Ação e aumentar os votos para 1500 (contra vontade), dando um total de 754 resultados.
Adicionei o campo país de produção.